In [1]:
import re
from langchain_community.document_loaders import PyPDFLoader

# 1. Wczytanie dokumentu
file_path = "last_unified_labor_code.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()

# 2. Łączenie wszystkich stron w jeden tekst i proste czyszczenie szumu
full_text = ""
for page in pages:
    content = page.page_content
    # Usuwanie stopki (©Kancelaria Sejmu + data + numer strony)
    # Regex dopasuje np. "©Kancelaria Sejmu s. 5/185" oraz datę pod spodem
    content = re.sub(r"©Kancelaria Sejmu.*s\.\s\d+/\d+", "", content)
    content = re.sub(r"2026-02-03", "", content) # usuwa datę generowania
    full_text += content + "\n"

# 3. Funkcja do podziału na artykuły
def split_into_articles(text):
    # Szuka wzorca: "Art. [liczba i ewentualnie litera]."
    # Używa (?=Art\.) aby zachować "Art." na początku każdego chunka
    pattern = r"(?=Art\.\s+\d+[a-z]*\.)"
    chunks = re.split(pattern, text)
    
    # Usuwa ewentualny pusty pierwszy element i białe znaki
    chunks = [c.strip() for c in chunks if c.strip()]
    return chunks

articles = split_into_articles(full_text)

# podgląd
print(f"Znaleziono {len(articles)} jednostek redakcyjnych (artykułów).")
print("\nPrzykład artykułu 1 (Art. 1):")
print(articles[0][:300])
print("\nPrzykład artykułu ze środka:")
print(articles[100][:300])

Znaleziono 477 jednostek redakcyjnych (artykułów).

Przykład artykułu 1 (Art. 1):
U S T AWA 
z dnia 26 czerwca 1974 r. 
Kodeks pracy1) 
Preambuła (uchylona) 
 
1) Niniejsza ustawa dokonuje w  zakresie swojej regulacji wdrożenia następujących dyrektyw 
Wspólnot Europejskich: 
1) dyrektywy 83/477/EWG z dnia 19 września 1983 r. w sprawie ochrony pracowników przed 
ryzykiem związanym

Przykład artykułu ze środka:
Art. 612. § 1. Odszkodowanie, o  którym mowa w  art. 611, przysługuje 
w wysokości wynagrodzenia pracownika za okres wypowiedzenia. W  przypadku 
rozwiązania umowy o  pracę zawartej na czas określony, odszkodowanie 
przysługuje w wysokości wynagrodzenia za czas, do którego umowa miała trwać, 
nie wi


In [5]:
processed_data = []

for i, chunk in enumerate(articles):
    # 1. Wyciąga numer artykułu z początku tekstu (np. "Art. 1.") ## szuka wzorca na samym początku chunka
    match = re.search(r"Art\.\s+(\d+[a-z]*)", chunk)
    
    if match:
        art_number = f"Art. {match.group(1)}"
    else:
        # jeśli to pierwszy chunk (ten z tytułem i preambułą)
        art_number = "Wstęp/Tytuł"

    # 2. Tworzy słownik z metadanymi
    metadata = {
        "art_id": art_number,
        "source": "Kodeks Pracy",
        "status_date": "2026-02-03",
        "chunk_id": i
    }
    
    # 3. Dodajemy do listy gotowy obiekt
    processed_data.append({
        "content": chunk,
        "metadata": metadata
    })

# test
print(f"Przygotowano {len(processed_data)} obiektów z metadanymi.")
print(f"Przykładowy wpis (indeks 5):")
print(f"Metadane: {processed_data[5]['metadata']}")
print(f"Treść (początek): {processed_data[5]['content'][:100]}...")

Przygotowano 477 obiektów z metadanymi.
Przykładowy wpis (indeks 5):
Metadane: {'art_id': 'Art. 4', 'source': 'Kodeks Pracy', 'status_date': '2026-02-03', 'chunk_id': 5}
Treść (początek): Art. 4. (uchylony)...
